<a href="https://colab.research.google.com/github/NonKhuna/DataPro-Algorithm/blob/main/Datapro_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab is a program that was implemented according to https://www.aaai.org/Papers/AAAI/2000/AAAI00-093.pdf. 

# **Import**

## library

In [ ]:
import numpy as  np
import pandas as pd
import json
import re

from queue import Queue

import math
from scipy.stats import norm
import ipywidgets as widgets

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Preprocess Data

### Synthesize Data
create Data field from column concatination.
Ex. "(-149.4574493, 61.4079146), 20757 OBERG RD, 99567"

In [ ]:
# download Data. Copy from https://www.kaggle.com/datasets/openaddresses/openaddresses-us-west
!gdown 1q9hiop-8aWnqfxSHX-s4fWgAU1aXsZ37

Downloading...
From: https://drive.google.com/uc?id=1q9hiop-8aWnqfxSHX-s4fWgAU1aXsZ37
To: /content/ak.csv
100% 20.9M/20.9M [00:00<00:00, 106MB/s] 


In [ ]:
df = pd.read_csv('ak.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,4,5,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,-149.362137,61.455872,26101,OLD GLENN HWY,TOWER,NaN,NaN,NaN,99567,NaN,95223943e9d286f2
1,-149.234243,61.441350,34709,EKLUTNA LAKE RD,NaN,NaN,NaN,NaN,99567,NaN,a736e3cf617bca0b
2,-149.211979,61.430535,35851,EKLUTNA LAKE RD,NaN,NaN,NaN,NaN,99567,NaN,7835b7a8e22af7a1
3,-149.212491,61.429942,35845,EKLUTNA LAKE RD,NaN,NaN,NaN,NaN,99567,NaN,58d5678c231cfc40
4,-149.273845,61.453997,32551,EKLUTNA LAKE RD,NaN,NaN,NaN,NaN,99567,NaN,0358d1b7a2b39d1f


In [ ]:
# drop column DISTRICT, ID and HASH
df2 = df.drop(['DISTRICT',"ID", "HASH"], axis=1)

# fill null value with ""
df2.fillna("", inplace = True)

df2.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,REGION,POSTCODE
0,-149.362137,61.455872,26101,OLD GLENN HWY,TOWER,,,99567
1,-149.234243,61.441350,34709,EKLUTNA LAKE RD,,,,99567
2,-149.211979,61.430535,35851,EKLUTNA LAKE RD,,,,99567
3,-149.212491,61.429942,35845,EKLUTNA LAKE RD,,,,99567
4,-149.273845,61.453997,32551,EKLUTNA LAKE RD,,,,99567


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292362 entries, 0 to 292361
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   LON       292362 non-null  float64
 1   LAT       292362 non-null  float64
 2   NUMBER    292362 non-null  object 
 3   STREET    292362 non-null  object 
 4   UNIT      292362 non-null  object 
 5   CITY      292362 non-null  object 
 6   REGION    292362 non-null  object 
 7   POSTCODE  292362 non-null  object 
dtypes: float64(2), object(6)
memory usage: 17.8+ MB


In [ ]:
# concate data test 1
# ref: https://www.indeed.com/career-advice/career-development/how-to-write-an-address#:~:text=Here's%20how%20to%20complete%20their,code%20on%20the%20final%20line.
# ex: (-149.362137, 61.4558721), 26101 OLD GLENN HWY, TOWER, 99567
def tranform_data(x):
  t_data = "(" + (x['LON']) + ", " + (x["LAT"]) + ")"
  t_data += (", "+ x["NUMBER"]) if x["NUMBER"]!="" else ""
  t_data += (" " + x["STREET"]) if x["STREET"]!="" else ""
  t_data += (", "+ x["UNIT"]) if x["UNIT"]!="" else ""
  t_data += (", "+ x["CITY"]) if x["CITY"]!="" else "" 
  if x["POSTCODE"]!="" :
    if x["REGION"]!="" :
       t_data += ", "+ x["REGION"] + " " + x["POSTCODE"]
    else :
       t_data += ", "+ x["POSTCODE"] 
  else :
    t_data += ", "+ x["REGION"]
  return t_data

data_field = df2.astype(str)
data_field = data_field.apply(lambda x: tranform_data(x), axis=1)


In [ ]:
# POSTCODE + CITY test 2
def tranform_data2(x):
  t_data = ""
  t_data += str(x["POSTCODE"]) + " " + x["CITY"]
  return t_data

df3 = df[["CITY","POSTCODE"]]
df3 = df3.dropna()
df3 = df3.reset_index()
data_field = df3.astype(str)
data_field = data_field.apply(lambda x: tranform_data2(x), axis=1)

### Tokenize Data field

In [ ]:
#@title Sample Size
from nltk.tokenize import word_tokenize
sample_size =  100#@param {type:"integer"}
if sample_size > len(data_field) : 
  print("Out of range!!!")
  print("Resize to", len(data_field))
  sample_size = len(data_field)

data_sample = data_field[:sample_size]

In [ ]:
# add new structure
ls = []
for x in range(80):
  ls.append("test "+ str(10000+int(x/10)))
data_sample = pd.concat([data_sample, pd.Series(ls)], ignore_index=True)
data_sample

0      99639.0 NINILCHIK
1      99639.0 NINILCHIK
2      99639.0 NINILCHIK
3      99639.0 NINILCHIK
4      99639.0 NINILCHIK
             ...        
175           test 10007
176           test 10007
177           test 10007
178           test 10007
179           test 10007
Length: 180, dtype: object

In [ ]:
seq_token_data = data_sample.apply(lambda x: word_tokenize(x)) # list of list
print("example data: [{}] ".format(seq_token_data[0]))
print("size: ", len(seq_token_data))
out = pd.DataFrame({"observation":[]})
out["observation"] = data_sample
out.to_csv("observation.csv")

example data: [['99639.0', 'NINILCHIK']] 
size:  180


# **Program**

### Significant fucntion


In [ ]:
def significant(count1, count2, num_token, token_count, alpha):
  """
    param:
      - count1 : count seq that token C appear after seq Q
      - count2 : count seq Q
      - num_token : amount of token C
      - token_count : amount of all token in data field
      - alpha 
  """
  p1 = count1 / count2
  p2 = num_token / token_count # overall prob of occurrence token T 
  p_a = (count1 + num_token) / (count2 + token_count)
  # print(count1, count2, num_token, token_count, alpha)
  # print(p_a ,(1-p_a) ,(1/count2 + 1/token_count), p_a * (1-p_a) * (1/count2 + 1/token_count))
  try :
    z = (p1 -p2) / math.sqrt(p_a * (1-p_a) * (1/count2 + 1/token_count))
  except :
    z = 1e9 # case prop = 1
  cdf = norm.cdf(-z, 0, 1) # z-score
  return cdf < alpha

def significant_2(n1,n, num_token,token_count, alpha) :
  p = num_token/token_count # overall prob of occurrence token T 
  mu = n * p
  sigma = math.sqrt(n*p*(1-p))
  z = (n1-mu)/(sigma/math.sqrt(n))
  cdf = norm.cdf(-z)
  return cdf < alpha

### Other function

In [ ]:
def get_dic_token(data, tree_type):
  # data is a list of list
  dictT = {}
  cnt = 0
  for row in data :
    # all_tokenT = set()
    for token in row :
      tokenTs = assign_type(token, tree_type, False)
      tokenTs.add(token)
      # all_tokenT = all_tokenT.union(tokenTs)
      for tokenT in tokenTs :
        if tokenT not in dictT :
          dictT[tokenT] = 1
        else :
          dictT[tokenT] +=1
      cnt+=1
    # for x in all_tokenT :
    #   if x not in dictT :
    #     dictT[x] =1
    #   else :
    #     dictT[x] +=1
  return dictT,cnt

def print_space(k) :
  for i in range(len(k)) :
    if k[i][1]:
      print(" ", end="")
    else :
      print("|", end="")
    for j in range(k[i][0]):
      print(" ", end="")

def display_tree_type(root="TOKEN", level=0, stack_space=[]):
  """
    show typs's tree structure display.
  """
  if level == 0:
    print("---------- Tree type -------------")
    print("TOKEN")
  cnt = 0
  size = len(tree_type[root]['children'])
  for child in tree_type[root]['children'] :
    print_space(stack_space)
    if cnt == size-1 :
      print("└── " + child)
    else :
      print("├── " + child)
    temp = stack_space.copy()
    temp.append([2 + int(len(child)/2), cnt == size-1])
    display_tree_type(child, level+1, temp)
    cnt+=1
  


def follow_by(token, tokenT, tree_type):
  return re.search(tree_type[tokenT]["regex"], token)

def sub_generalize(token1,token2, tree_type): # O(1)
  # token2 is a generalization of token1
  dif_level = abs(ord(tree_type[token1]["code"]["key"]) - ord(tree_type[token2]["code"]["key"]))
  return tree_type[token2]["code"]["data"] in tree_type[token1]["code"]["data"] and dif_level==1

In [ ]:
c_count = 1
q_count = 10000
token_t = 1
all_t = 10000*2
alpha_t = 0.05
print("sig1:",significant(c_count, q_count, token_t, all_t, alpha_t))
print("sig2:",significant_2(c_count, q_count, token_t, all_t, alpha_t))

sig1: False
sig2: True


### Assign Token's type function


In [ ]:
# download Tree of type, json format
!gdown 1LIGUjJRCdjV_xh3WzFPEqOgogQNokt16

Downloading...
From: https://drive.google.com/uc?id=1LIGUjJRCdjV_xh3WzFPEqOgogQNokt16
To: /content/tree-type.json
100% 1.40k/1.40k [00:00<00:00, 2.33MB/s]


In [ ]:
def load_tree_type(path):
  '''
    Args:
      path (str): path where json file store.
    Returns:
      Dict: tree represent.
  '''
  with open(path, 'r') as f :
    tree = json.load(f)
  return tree

def fix_format(token):
  return "^"+re.escape(token)+"$"

def revese_escape(token) :
  return re.sub(r'\\(.)', r'\1', token)


def assign_type(token, tree_type, grows_tree=False) :
  # assign token's type and continue to add special token to the tree
  
  types = set()
  last_node = "TOKEN" # last node that matching.

  q = Queue()
  for k in tree_type["TOKEN"]["children"] :
    q.put(k)

  while not q.empty() :
    now_type = q.get()
    if re.search(tree_type[now_type]["regex"], revese_escape(token)) :
      types.add(now_type)
      last_node = now_type
      for k in tree_type[now_type]["children"] :
        q.put(k)
  
  if  tree_type[last_node]["regex"] != fix_format(last_node) and (token not in tree_type) and grows_tree: # general node
      # constuct new special node
      tree_type[token] = dict()
      tree_type[token]["regex"] = fix_format(token)
      tree_type[token]["children"] = []
      tree_type[token]["parent"] = last_node
      tree_type[token]["code"] = dict()
      tree_type[token]["code"]["key"] = chr(ord(tree_type[last_node]["code"]["key"]) + 1)
      tree_type[token]["code"]["data"] = tree_type[last_node]["code"]["data"] + "." + tree_type[token]["code"]["key"] + str(len(tree_type[last_node]["children"]))
      tree_type[last_node]["children"].append(token)
      types.add(token)

  return types

In [ ]:
tree_type = load_tree_type("tree-type.json")
display_tree_type()

---------- Tree type -------------
TOKEN
├── PUNCT
└── ALPHANUM
       ├── ALPHA
       |    ├── CAPS
       |    ├── LOWER
       |    └── ALLCAPS
       └── NUMBER


In [ ]:
assign_type("Nectec", tree_type)

{'ALPHA', 'ALPHANUM', 'CAPS'}

In [ ]:
# test
ex = tree_type['ALPHA']['regex']
text = 'ttest'
x = re.search(ex, text)
print(x)

<re.Match object; span=(0, 5), match='ttest'>


## Object Class

In [ ]:
class node:
  def __init__(self, token, examples, pattern, next_po,parent):
    self.token = token
    self.examples = examples
    self.count = len(examples)
    self.pattern = pattern
    self.children = []
    self.next_po = next_po
    self.parent = parent

  def info(self):
    print("token:", self.token)
    print("amount of example:", self.count)
    print("pattern:", self.pattern)
    print("next_po:", self.next_po)

  def get_children_token_type(self, tree_type, dict_token, k):
    uniq_token = set()
    for ex in self.examples :
      # bound range
      if len(ex) > self.next_po :
          uniq_token.add(ex[self.next_po])
    
    set_token_type = set()
    for token in uniq_token :
      is_uniq = (dict_token[token] >= k)
      # print(token, dict_token[token], k, is_uniq)
      set_token_type = set_token_type.union(assign_type(token, tree_type, is_uniq))
    return set_token_type
  
  def print_space(self, k) :
    for i in range(len(k)) :
      if k[i][1]:
        print(" ", end="")
      else :
        print("|", end="")
      for j in range(k[i][0]):
        print(" ", end="")

  def display(self, level=0, stack_space=[]):
    cnt = 0
    size = len(self.children)
    for child in self.children :
      self.print_space(stack_space)
      if cnt == size-1 :
        print("└── " + child.token)
      else :
        print("├── " + child.token)
      temp = stack_space.copy()
      temp.append([2 + int(len(child.token)/2), cnt == size-1])
      child.display(level+1, temp)
      cnt+=1
    
  def find(self,name_token) :
    for child in self.children :
      if child.token == name_token : return child
      res = child.find(name_token) 
      if res : return res
    return None


    

## Main

In [ ]:
# define constant
alpha = 0.05
k_percentage = 5 #appears in at least k example, in term of percent.
k = int((k_percentage/100) * len(seq_token_data)) 

# load tree type
tree_type = load_tree_type("tree-type.json")

# root node
q = Queue()
root = node("root", list(seq_token_data), [], 0, None)
q.put(root)

# Knowing how many token occuring in the data before assigned type.
# ** token_count required before all seq token's type are generated. Actualy token
dict_token, token_count = get_dic_token(seq_token_data, tree_type)

# tempo-------------------------------------------------------------
s2 = {"before_pattern_Q":[], "amount_Q":[], "next_token":[], "amount_C":[], "prop_(Q,C)":[],"prop_(next_token)":[], "all_token":[], "current_sig_predict":[]}
# temp ---------------------------------------------------------------



while not q.empty():
    # node Q
    Q = q.get()

    # Create children Of Q 
    # Get all posible token that are children of Q
    all_token_type = Q.get_children_token_type(tree_type, dict_token, k)
    for tokenT in all_token_type :
      C = node(tokenT, [], Q.pattern.copy(), Q.next_po+1, Q)
      C.examples = [ex for ex in Q.examples if (Q.next_po < len(ex) and follow_by(ex[Q.next_po],tokenT, tree_type))]
      C.count = len(C.examples)
      C.pattern.append(tokenT)
      # temp ---------------------------
      s2["before_pattern_Q"].append(Q.pattern)
      s2["amount_Q"].append(Q.count)
      s2["amount_C"].append(C.count)
      s2["next_token"].append(tokenT)
      s2["prop_(Q,C)"].append(C.count / Q.count)
      s2["all_token"].append(token_count)
      s2["prop_(next_token)"].append(dict_token[tokenT]/ token_count)
      sig = significant(C.count, Q.count, dict_token[tokenT], token_count, alpha)
      s2["current_sig_predict"].append(sig)
      # temp ---------------------------
      if significant(C.count, Q.count, dict_token[tokenT], token_count, alpha) :
        Q.children.append(C)
    
  
    # Prune gerneralizations ?
    temp_children = []
    for C in Q.children :
      N = C.count
      for S in Q.children :
        if S.token != C.token and sub_generalize(S.token, C.token, tree_type):
          N -= S.count
      if significant(N, Q.count, dict_token[C.token], token_count, alpha) :
        temp_children.append(C)
        q.put(C)
    Q.children = temp_children

    
    
    # Determinize children of Q
    for C in Q.children :
      for S in Q.children :
        if S.token != C.token and sub_generalize(S.token, C.token, tree_type) :
          new_examples = []
          for x in C.examples :
            if x not in S.examples :
              new_examples.append(x)
          C.examples = new_examples
      C.count = len(C.examples)

# temp ---------------------------
out = pd.DataFrame.from_dict(s2)
out.to_csv("prop_respect_to_occure_observation.csv")
# temp ---------------------------

In [ ]:
root.display()

├── ALPHANUM
|      ├── ALPHANUM
|      ├── NINILCHIK
|      └── KALIFORNSKY
├── test
|    ├── 10003
|    ├── 10001
|    ├── 10005
|    ├── 10006
|    ├── 10004
|    ├── 10007
|    ├── 10002
|    └── 10000
├── 99639.0
|     └── NINILCHIK
└── 99669.0
      └── KALIFORNSKY


In [ ]:
display_tree_type()

---------- Tree type -------------
TOKEN
├── PUNCT
└── ALPHANUM
       ├── ALPHA
       |    ├── CAPS
       |    ├── LOWER
       |    |    └── test
       |    └── ALLCAPS
       |          ├── NINILCHIK
       |          └── KALIFORNSKY
       └── NUMBER
             ├── 99639.0
             ├── 99669.0
             ├── 10003
             ├── 10001
             ├── 10005
             ├── 10006
             ├── 10004
             ├── 10007
             ├── 10002
             └── 10000


In [ ]:
s= root.find("ALPHA")
if s :
  s.info()

In [ ]:
# Extract patterns from tree
res = []
q = Queue()
q.put(root)
while not q.empty() :
  Q = q.get()
  for C in Q.children :
    N = C.count
    q.put(C)
    for S in C.children :
        N-=S.count
    if significant(N, Q.count, dict_token[C.token], token_count, alpha) :
      res.append(C.pattern)

for x in res :
  print("[{}]".format(x))
print(len(res))

[['ALPHANUM', 'ALPHANUM']]
[['ALPHANUM', 'NINILCHIK']]
[['ALPHANUM', 'KALIFORNSKY']]
[['test', '10003']]
[['test', '10001']]
[['test', '10005']]
[['test', '10006']]
[['test', '10004']]
[['test', '10007']]
[['test', '10002']]
[['test', '10000']]
[['99639.0', 'NINILCHIK']]
[['99669.0', 'KALIFORNSKY']]
13
